# Ch 7 Prophet and Covid new cases predictions

In [1]:
#pip install greykite
#Silverkite is LinkedIn's time series algorithm
#greykite is the library
# BUT CAN also use Prophet for a library

In [2]:
# data from csv("https://covid.ourworldindata.org/data/owidcovid-data.csv") 
# data starts on Jan 1, 2020 and has information about global covid cases
# I chopped the locations down to US only

In [3]:
import pandas as pd
from fbprophet import Prophet

covid = pd.read_csv("C:/Users/hp/Documents/Data Science/Machine Learning Time-Series book/data/covid_Jan_1_2020_to_Jan_9_2022.csv")

In [4]:
covid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2/24/2020,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2/25/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2/26/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2/27/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2/28/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [5]:
covid["date"] = pd.to_datetime(covid["date"])

In [6]:
covid.location.unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Asia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethi

In [7]:
#To concentrate on U.S. cases
df = covid[covid.location == "United States"].set_index("date", drop=True).resample('D').interpolate(method='linear')

In [8]:
from greykite.framework.templates.autogen.forecast_config import (ForecastConfig, MetadataParam)


#creating a time column called 'date', a value column called "new_cases" and a daily frequency
metadata = MetadataParam(
    time_col="date",
    value_col="new_cases",
    freq="D"
)

In [10]:
#create forecast object and stores the results
#forecasting 90 days ahead with a prediction interval of 95%
import warnings
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum

forecaster = Forecaster()

warnings.filterwarnings("ignore", category=UserWarning)
result = forecaster.run_forecast_config(
    df=df.reset_index(),
    config=ForecastConfig(
        model_template=ModelTemplateEnum.PROPHET.name,
        forecast_horizon=90,
        coverage=0.95,
        metadata_param=metadata,
    )
)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\hp\anaconda3\lib\site-packages\greykite\sklearn\transform\null_transformer.py:177: RuntimeWarning: Input data has many null values. Missing 11.14% of one input.
  warn

In [12]:
forecast = result.forecast
forecast.plot().show()

In [13]:
#to get upper and lower confidence intervals of the forecasts
forecast.df.head().round(2)

,date,actual,forecast,forecast_lower,forecast_upper
0,2020-01-23,0.0,-5977.90,-113725.73,94551.40
1,2020-01-24,1.0,13723.02,-82748.90,105097.39
2,2020-01-25,0.0,-17746.40,-113847.23,85531.28
3,2020-01-26,3.0,-11128.42,-112714.94,89019.19
4,2020-01-27,0.0,-2982.96,-95918.22,93221.17


In [14]:
#performance of the historical forecast on the holdout test set
from collections import defaultdict

backtest = result.backtest
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
    backtest_eval[metric].append(value)
    backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics.head()

,train,test
CORR,0.890939,0.665305
R2,0.787472,0.242704
MSE,955594534.14728,31342708890.860096
RMSE,30912.692121,177038.721445
MAE,21113.73296,125937.568241


In [15]:
#our model to new data
model = result.model
future_df = result.timeseries.make_future_dataframe(
    periods=4,
    include_history=False
)
model.predict(future_df)

C:\Users\hp\anaconda3\lib\site-packages\greykite\sklearn\transform\null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 100.00% of one input.



,ts,forecast,forecast_lower,forecast_upper
0,2022-01-10,193427.290308,93438.142640,287585.596807
1,2022-01-11,185167.818004,86137.356337,290200.182583
2,2022-01-12,187569.580147,98116.711269,287128.194142
3,2022-01-13,201770.899697,95052.884174,298649.324724
